# Importer

In [21]:
import pandas as pd
import os
import re
import numpy as np

In [12]:
file_path = '~/Desktop/projet_la/bdd/bdd.csv'

# Essai avec l'encodage 'latin1'
df = pd.read_csv(file_path, sep=';', encoding='UTF-8', low_memory=False)

df.head()

,idVisit,visitorId,fingerprint,type (actionDetails 0),url (actionDetails 0),pageTitle (actionDetails 0),pageIdAction (actionDetails 0),idpageview (actionDetails 0),serverTimePretty (actionDetails 0),pageId (actionDetails 0),...,revenueSubTotal (actionDetails 32),revenueTax (actionDetails 32),revenueShipping (actionDetails 32),revenueDiscount (actionDetails 32),items (actionDetails 32),productViewPrice (actionDetails 6),productViewPrice (actionDetails 9),productViewPrice (actionDetails 17),productViewPrice (actionDetails 0),productViewPrice (actionDetails 8)
0,2143,7ffb5e14a0ae6e4d,e6167018f5abd82e,action,https://educapacites.fr/447-2/,Concentration | Éducapacités,6,lZ8BSi,30 sept. 2024 22:28:13,7887,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2142,014c2c3d42c4eb72,333d9ccae11a3eb6,action,https://educapacites.fr/,"Éducapacités | Pour ton bien-être émotionnel, ...",2,PVoZYy,30 sept. 2024 20:29:06,7884,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2141,a6fbcefff5113692,ef3d594bc6dded43,action,https://educapacites.fr/,"Éducapacités | Pour ton bien-être émotionnel, ...",2,xCTaDo,30 sept. 2024 18:23:12,7883,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2140,c59617c719e24e0c,e88f1ffd961857b8,action,https://educapacites.fr/,"Éducapacités | Pour ton bien-être émotionnel, ...",2,E4U9UW,30 sept. 2024 18:09:58,7882,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2138,5f3f8420f61bfd55,badfe6a9b403694f,action,https://educapacites.fr/mon-espace/,Mon espace | Éducapacités,95,Gf4cQZ,30 sept. 2024 14:39:48,7866,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Chemin type

## Classement des pageTitle

In [18]:
# Identifier les colonnes "pageTitle"
page_title_columns = [col for col in df.columns if 'pageTitle (actionDetails' in col]

# Concaténer toutes ces colonnes en une seule série
all_page_titles = pd.concat([df[col] for col in page_title_columns], ignore_index=True)

# Calculer la fréquence de chaque pageTitle 
page_title_counts = all_page_titles.value_counts()

# Créer une correspondance en triant par fréquence décroissante
# Le titre le plus fréquent reçoit le numéro 1, le suivant 2, etc.
mapping = {title: i+1 for i, title in enumerate(page_title_counts.index)}

# Créer un DataFrame de concordance incluant le titre, son numéro unique et son nombre d'occurrences
concordance_df = pd.DataFrame({
    'pageTitle': page_title_counts.index,
    'Unique Number': [mapping[title] for title in page_title_counts.index],
    'Count': page_title_counts.values
})

# Générer la table HTML depuis le DataFrame
html_table = concordance_df.to_html(index=False)

# Construire le contenu HTML 
html_output = f"""<!DOCTYPE html>
<html lang="fr">
<head>
  <meta charset="utf-8">
  <title>Tableau de concordance des pageTitles</title>
</head>
<body>
  {html_table}
</body>
</html>
"""

# Enregistrer le fichier HTML sur le Bureau de votre MacBook
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
file_path = os.path.join(desktop_path, "concordance.html")
with open(file_path, 'w', encoding='utf-8') as f:
    f.write(html_output)

print(f"Le fichier HTML a été enregistré sur le Bureau à l'emplacement : {file_path}")

Le fichier HTML a été enregistré sur le Bureau à l'emplacement : /Users/rezisabashvili/Desktop/concordance.html


## Chemin

In [25]:
# Identifier les colonnes contenant "pageTitle (actionDetails" dans le DataFrame
page_title_columns = [col for col in df.columns if 'pageTitle (actionDetails' in col]

# Fonction pour extraire le numéro d'actionDetails depuis le nom de la colonne
def extract_action_index(col_name):
    m = re.search(r'actionDetails\s*(\d+)', col_name)
    return int(m.group(1)) if m else -1

# Trier les colonnes par ordre croissant du numéro d'actionDetails
sorted_page_title_cols = sorted(page_title_columns, key=extract_action_index)

# Créer le mapping : on concatène toutes les colonnes pour obtenir toutes les valeurs uniques et leur fréquence
all_page_titles = pd.concat([df[col] for col in sorted_page_title_cols], ignore_index=True)
page_title_counts = all_page_titles.value_counts()  # tri décroissant par défaut
mapping = {title: i+1 for i, title in enumerate(page_title_counts.index)}

# Fonction pour extraire le chemin d'une ligne
def get_path(row):
    """
    Parcourt les colonnes triées "pageTitle (actionDetails X)".
    Ajoute dans le chemin le numéro unique associé à chaque titre.
    S'arrête dès qu'une valeur est manquante.
    """
    path = []
    for col in sorted_page_title_cols:
        value = row[col]
        if pd.isna(value) or value == "":
            break  # arrêt dès qu'il n'y a plus de valeur
        path.append(mapping.get(value, None))
    return tuple(path)

# Appliquer la fonction sur chaque ligne pour créer la colonne 'path'
df['path'] = df.apply(get_path, axis=1)

# Enregistrer le nombre d'actionDetails réalisés (longueur du chemin)
df['action_details_count'] = df['path'].apply(len)

# Regrouper les chemins identiques et compter leur nombre d'occurrences
path_counts = df['path'].value_counts().reset_index()
path_counts.columns = ['Chemin', 'Occurrences']

# Ajouter une colonne indiquant le nombre d'actions (la longueur du tuple)
path_counts['Nombre d\'actions'] = path_counts['Chemin'].apply(len)

# Réorganiser les colonnes pour la sortie finale
result_df = path_counts[['Chemin', 'Nombre d\'actions', 'Occurrences']]

# Générer le tableau HTML à partir du DataFrame
html_table = result_df.to_html(index=False)

# Construire le contenu HTML complet avec la balise meta pour UTF-8
html_output = f"""<!DOCTYPE html>
<html lang="fr">
<head>
  <meta charset="utf-8">
  <title>Chemins et Fréquences</title>
</head>
<body>
  {html_table}
</body>
</html>
"""

# Enregistrer le fichier HTML sur le Bureau de votre MacBook
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
output_file = os.path.join(desktop_path, "chemins.html")
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(html_output)

print(f"Le fichier HTML a été enregistré sur le Bureau à l'emplacement : {output_file}")

Le fichier HTML a été enregistré sur le Bureau à l'emplacement : /Users/rezisabashvili/Desktop/chemins.html
